# 1. Dataset Experiments
**Author:** Christian Byron  **Date:** 15-Aug-21

This notebook documents experiements to validate the correct loading of the KTH dataset using the Lightning DataModule class approach, and to compare performance of techniques for loading and caching the video data for the purposes of running on GPUs during Neural Net training and validation.

### Profiling Task 1 - Test performance of loading select frames from video

In [1]:
from profiler import profile
import imageio, cv2, torch
import numpy as np

video_file = 'C:/Users/s441606/Documents/Videos/KTHA/boxing/person01_boxing_d1_uncomp.avi'
video_len = 330

def get_frames_using_image_io(start, end):
    frames = []
    vid = imageio.get_reader(video_file,  'ffmpeg')
    for frame_num in range(start, end):
        frame = vid.get_data(frame_num)
        frames.append(frame)
    return frames

def get_frames_using_cv(start, end):
    frames = []
    vid = cv2.VideoCapture(video_file)
    for frame_num in range(start, end):
        vid.set(1, frame_num)
        ret, frame = vid.read()
        frames.append(frame)
    vid.release()
    return frames


@profile
def first_code_to_profile():
    instances = []
    for i in range(0,100):
        start = (i * 15) % video_len
        #frames = get_frames_using_image_io(start, start+15)
        frames = get_frames_using_cv(start, start+15)
        frames = torch.from_numpy(np.moveaxis( np.array(frames), 1,0)) / 255
        instances.append(frames)
        
    return instances
         
%time instances = first_code_to_profile()

         7202 function calls in 13.575 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.144    0.144   13.575   13.575 <ipython-input-1-871f13844e8a>:27(first_code_to_profile)
      100    0.249    0.002   13.412    0.134 <ipython-input-1-871f13844e8a>:16(get_frames_using_cv)
     1500   12.818    0.009   12.818    0.009 {method 'set' of 'cv2.VideoCapture' objects}
     1500    0.181    0.000    0.181    0.000 {method 'read' of 'cv2.VideoCapture' objects}
      100    0.163    0.002    0.163    0.002 {method 'release' of 'cv2.VideoCapture' objects}
      100    0.013    0.000    0.013    0.000 {built-in method numpy.array}
      100    0.000    0.000    0.004    0.000 <__array_function__ internals>:2(moveaxis)
      100    0.000    0.000    0.004    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      100    0.001    0.000    0.004    0.000 C:\Users\s441606\.conda\envs\pytorch\

Results using image.io:
   **162056 function calls in 12.528 seconds**
   
`   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3405    0.022    0.000   12.284    0.004 ... packages\imageio_ffmpeg\_io.py:66(read_frames)
      741   11.061    0.015   11.061    0.015 {built-in method time.sleep}
      100    0.001    0.000    5.697    0.057 ... packages\imageio\core\functions.py:148(get_reader)
`    
Results using cv2:
   **7202 function calls in 8.232 seconds**

`   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1500    7.700    0.005    7.700    0.005 {method 'set' of 'cv2.VideoCapture' objects}
     1500    0.113    0.000    0.113    0.000 {method 'read' of 'cv2.VideoCapture' objects}
`

### Profiling Task 2 - Saving resulting data to file

In [2]:
@profile
def second_code_to_profile():
    for i in range(0,100):
        frames = instances[i]
        torch.save(frames, 'data/frames{}.pt'.format(i))
    
%time second_code_to_profile()

         9502 function calls in 0.552 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.552    0.552 <ipython-input-2-9ca5293dbccd>:1(second_code_to_profile)
      100    0.002    0.000    0.550    0.006 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:333(save)
      100    0.464    0.005    0.476    0.005 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:457(_save)
      100    0.000    0.000    0.047    0.000 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:228(_open_file_like)
      100    0.000    0.000    0.046    0.000 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:210(__init__)
      100    0.046    0.000    0.046    0.000 {built-in method io.open}
      100    0.000    0.000    0.019    0.000 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.p

Results: **9502 function calls in 0.367 seconds**
   
`   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     100    0.308    0.003    0.317    0.003 ...packages\torch\serialization.py:457(_save)
`    

### Profiling Task 3 - Loading data from file

In [3]:
@profile
def third_code_to_profile():
    instances = []
    for i in range(0,100):
        frames = torch.load('data/frames{}.pt'.format(i))
        instances.append(frames)
        
    return instances

%time third_code_to_profile()

         6602 function calls in 1.183 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    1.183    1.183 <ipython-input-3-820211cc5dd2>:1(third_code_to_profile)
      100    0.002    0.000    1.182    0.012 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:502(load)
      100    0.000    0.000    0.987    0.010 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:228(_open_file_like)
      100    0.000    0.000    0.987    0.010 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:210(__init__)
      100    0.986    0.010    0.986    0.010 {built-in method io.open}
      100    0.003    0.000    0.174    0.002 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:836(_load)
      100    0.003    0.000    0.168    0.002 {method 'load' of '_pickle.Unpickler' objects}
      100    0.000    0.000   

[tensor([[[[0.6745, 0.6745, 0.6745],
           [0.6784, 0.6784, 0.6784],
           [0.6863, 0.6863, 0.6863],
           ...,
           [0.6824, 0.6824, 0.6824],
           [0.6745, 0.6745, 0.6745],
           [0.6863, 0.6863, 0.6863]],
 
          [[0.6745, 0.6745, 0.6745],
           [0.6784, 0.6784, 0.6784],
           [0.6824, 0.6824, 0.6824],
           ...,
           [0.6392, 0.6392, 0.6392],
           [0.6235, 0.6235, 0.6235],
           [0.6157, 0.6157, 0.6157]],
 
          [[0.6157, 0.6157, 0.6157],
           [0.6235, 0.6235, 0.6235],
           [0.6235, 0.6235, 0.6235],
           ...,
           [0.6314, 0.6314, 0.6314],
           [0.6275, 0.6275, 0.6275],
           [0.6196, 0.6196, 0.6196]],
 
          ...,
 
          [[0.6980, 0.6980, 0.6980],
           [0.6745, 0.6745, 0.6745],
           [0.6667, 0.6667, 0.6667],
           ...,
           [0.7059, 0.7059, 0.7059],
           [0.6863, 0.6863, 0.6863],
           [0.6784, 0.6784, 0.6784]],
 
          [[0.6863,

Results: **6602 function calls in 0.773 seconds**
   
`   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     100    0.647    0.006    0.647    0.006 {built-in method io.open}
` 

### Conclusion for DataModule / Dataset design

The above illustrates that loading data from the video files (where 1 file has multiple instances of 15 frames) is slower compared with loading from individual instance files. That is for 100 instances image.io library takes 12 secs, OpenCv takes 8secs and Torch.load takes 0.7 secs.

So the DataModule `prepare_data` function should call the dataset in order to load from videofile, and save the initial frames to a seperate directory where they will be loaded as needed by the `__get_item__` function


In [2]:
from profiler import profile
from KTH_DataModule import KTH_DataModule

dm = KTH_DataModule('C:/Users/s441606/Documents/Videos/KTHB')

@profile
def forth_code_to_profile():
    dm.prepare_data()
    for i, (images, label) in enumerate(dm.test_dataloader()):
        pass

%time forth_code_to_profile()

         12207 function calls (12157 primitive calls) in 5.271 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    5.271    5.271 <ipython-input-2-d1b19d30cbd5>:6(forth_code_to_profile)
        1    0.000    0.000    5.116    5.116 C:\Users\s441606\.conda\envs\pytorch\lib\site-packages\pytorch_lightning\core\datamodule.py:393(wrapped_fn)
        1    0.000    0.000    5.116    5.116 C:\Users\s441606\Documents\Code Projects\KTH-Action-Recognition\KTH_DataModule.py:82(prepare_data)
        1    0.000    0.000    5.116    5.116 C:\Users\s441606\Documents\Code Projects\KTH-Action-Recognition\KTH_DataModule.py:17(prepare_data)
        1    0.230    0.230    5.109    5.109 C:\Users\s441606\Documents\Code Projects\KTH-Action-Recognition\KTH_DataModule.py:48(__process_video_frames)
      885    4.600    0.005    4.600    0.005 {method 'set' of 'cv2.VideoCapture' objects}
       77    0.152    0.002    0

Results: **2807194 function calls (2797564 primitive calls) in 2241.217 seconds (37 mins)**
   
```
    ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    205725 1921.614    0.009 1921.614    0.009 {method 'set' of 'cv2.VideoCapture' objects}
     13715    0.342    0.000   96.970    0.007 ...packages\torch\serialization.py:333(save)
     13715   79.264    0.006   81.294    0.006 ...packages\torch\serialization.py:457(_save)
     18508   64.377    0.003   64.377    0.003 {built-in method io.open}
    205725   31.564    0.000   31.564    0.000 {method 'read' of 'cv2.VideoCapture' objects}
     13715   21.831    0.002   21.831    0.002 {method 'release' of 'cv2.VideoCapture' objects}
      4792   16.656    0.003   16.693    0.003 ...packages\torch\serialization.py:841(load_tensor)
```

https://nickmccullum.com/how-to-profile-python-code/